In [31]:
# dp practice (haven't used Python for a while since working on Julia)
cache = {1:1, 2:1}
def fib(n):
    if n in cache:
        return cache[n]
    current = fib(n-1) + fib(n-2)
    cache[n] = current
    return current

num = 100
print("fib(",num,"):",fib(num))
# print(cache)

# Ahh, it's as great as I remember.
    

fib( 100 ): 354224848179261915075


In [32]:
# Get pandas for dealing with xlsx
import pandas as pd

In [193]:
# Old numpy
import numpy as np
# Stats
import statistics as st
# Plotting
import matplotlib.pyplot as plt
import math

In [46]:
# set path
path = "/Users/chaseabram/UChiGit/Skwad/Empirical Analysis III/Problem Sets/Heckman Pset 1/PS1_Q5_Data.xlsx"
# read file
xls = pd.ExcelFile(path)

In [205]:
# Get number of sheets (72)
numSheets = len(xls.sheet_names)
# initialize
dfs = [0]*numSheets
betas = np.zeros((numSheets,3))
# read each sheet into data frame such that dfs[sheet][variable][observation]
for i in range(0,numSheets):
    # Add to data frame
    sheetNum = str(i)
    sheetName = 'DataSet' + sheetNum
    dfs[i] = pd.read_excel(xls, sheetName)
    # read in data
    X = np.zeros((3, len(dfs[i])))
    X[0] = [1]*len(dfs[i])
    X[1] = dfs[i]['X1']
    X[2] = dfs[i]['X2']
    Y = dfs[i]['Y']
    # run ols with beta1 and beta 1
    betas[i] = np.dot(np.linalg.inv(np.dot(X,np.transpose(X))),np.dot(X,Y))
    
    # estimate standard deviation (NEED TO FIX FOR BETA2)
#     print("mean",np.mean(X[2]))
    sigma = math.sqrt(np.var(X[2]))
#     print("var:",np.var(X[2]))
#     print("sd:",sigma)
    tstat = math.sqrt(len(dfs[i]))*betas[i][1]/sigma
    print(tstat)
    
    # find relevant t-statistic
#     tstat = betas[i][1]/
    
    
    
# np.dot(X,np.transpose(X))
# dataHist = np.histogram(betas[:,1])
# betas
# plt.hist(dataHist)

# dfs[0]["X1"][0]

9.223554200437155
8.740499021666624
16.532593708406576
9.862711657159947
9.059049974744092
8.14788683516036
12.563428175469127
12.620295358685592
6.9557763286225365
1.3774628020042539
9.798707487377241
11.803213928070724
9.575000607827228
21.89490156844744
53.08192381219956
2.118669328533758
11.462028126174324
4.4732790778425295
-41896073.371400945
-162695.7363343059
156200016.79048243
196580.99290631193
22100283.379923984
458835.97134344885
-36501657.21016834
2926866.673940093
-205692320.34397775
-41643.54437950463
-14104154.724990858
4922017.996907102
-422648581.9159778
9603837.368646877
17836332426.98756
-591720.0231757595
23172815.252343174
-3452023.9766265303
9.53447901230303
10.47804137101567
4.69788686862459
8.926125816918201
10.589583370156936
11.445277399457037
13.24587327489305
15.678564820514664
14.155392807885768
2.037555341969885
10.735805966647035
9.166936240606914
1.3665147865004568
-3.543909121383677
17.94241940428394
48.02890201328631
9.311160104145861
12.0541319320643